In [6]:
import sys 
import time
import cv2 as cv
import numpy as np
from audiolazy import str2midi #import function to convert note names to midi numbers (could also use: midi2str, str2freq, freq2str,freq2midi,midi2freq) https://pypi.org/project/audiolazy/
import pandas as pd   #import library for loading data, https://pypi.org/project/pandas/
import matplotlib.pylab as plt  #import library for plotting, https://pypi.org/project/matplotlib/
from midiutil import MIDIFile #import library to make midi file, https://midiutil.readthedocs.io/en/1.2.1/
from pythonosc import udp_client

In [7]:
ip = "127.0.0.1"
port = 5555

client = udp_client.SimpleUDPClient(ip, port)

In [3]:
note_names = []

# scale_selection = int(input('select scale: ( 1) 10~109 2) major 3) pentatonic 4) C lydian)'))
scale_selection = 1

# TODO: change to dictionary
# Option 1: using 100 consecutive notes
if(scale_selection == 1):
    for i in range(21, 21+88):
        note_names.append(i)
    isNoteStr = False

# 4 octaves of major scale
elif(scale_selection == 2):
    note_names = ['C2','D2','E2','F2','G2','A2','B2',
                 'C3','D3','E3','F3','G3','A3','B3',
                 'C4','D4','E4','F4','G4','A4','B4',
                 'C5','D5','E5','F5','G5','A5','B5']
    isNoteStr = True

#4 octaves of major pentatonic scale 
elif(scale_selection == 3):
    note_names = ['C2','D2','E2','G2','A2',
                 'C3','D3','E3','G3','A3',
                 'C4','D4','E4','G4','A4',
                 'C5','D5','E5','G5','A5']
    isNoteStr = True

#custom note set (a voicing of a Cmaj13#11 chord, notes from C lydian)
elif(scale_selection == 4):
    note_names = ['C1','C2','G2',
                 'C3','E3','G3','A3','B3',
                 'D4','E4','G4','A4','B4',
                 'D5','E5','G5','A5','B5',
                 'D6','E6','F#6','G6','A6']
    isNoteStr = True

else: 
    print("please select available scale")
# print(note_names)
# note_names = ['C1','D1','E1','F1','G1','A1','B1',
#               'C2','D2','E2','F2','G2','A2','B2',
#              'C3','D3','E3','F3','G3','A3','B3',
#              'C4','D4','E4','F4','G4','A4','B4',
#              'C5','D5','E5','F5','G5','A5','B5',
#              'C6','D6','E6','F6','G6']
# isNoteStr = True

if(isNoteStr):
    # print(note_names)
    note_midis = [str2midi(n) for n in note_names] #make a list of midi note numbers 
else:
    note_midis = note_names
# print(note_midis)

num_notes = len(note_midis)
print('Resolution:', num_notes, 'notes')

Resolution: 88 notes


In [16]:
printFlag = True   # for debugging process
testFlag = True

if(testFlag):
    # video = "lp1.mov"
    video = "test.mp4"
    # cap = cv.VideoCapture(video)
    cap = cv.VideoCapture("./video/" + video)
else:
    video = str(input('Video to use: '))
    if (video == "rt"): 
        video = int(0)

    cap = cv.VideoCapture(video)

In [17]:
### image preprocessing ###
# w = 1
# h = 480 * 2
# y = 1080 // 2 - w // 2
# x = 1920 // 2 - h // 2

# ret = a boolean return value from getting the frame, first_frame = the first frame in the entire video sequence
ret, first_frame = cap.read()   # read the very first frame
print("video size in pixel: ", np.shape(first_frame))
if(testFlag):
    isManual = bool(False)
else:
    isManual = str(input('Select ROI in manual? (y/n):'))
    if (isManual == "y"):
        isManual = bool(True)
    else:
        isManual = bool(False)

if(isManual):
    # if want to select area manually
    x, y, w, h = cv.selectROI('mouse', first_frame, False)
    print("x_pos, y_pos: ", x, y)
    print("width, height: ", w, h)
    cv.waitKey(1)
    cv.destroyWindow('mouse')
    cv.destroyAllWindows()
    cv.waitKey(1)
else:
    x, y, w = np.shape(first_frame)[0] // 2, 50, 0
    if(np.shape(first_frame)[1] < 2000):
        h = 88*5
    else:
        h = 88*10
    # x, y, w, h = 1080, 75, 0, 1000

video size in pixel:  (2118, 2068, 3)


In [18]:
frame_cnt = 0
# bound = int(input('bound:'))
bound = 0
# leap = int(input('leap: '))
leap = 3
tp = 60

if h % num_notes == 0:
    num_area = (h // num_notes) 
else:
    num_area = (h // num_notes) + 1
# print(num_area)

if(printFlag):
    print("first_frame shape: ", np.shape(first_frame))
    print("x: ", x)
    print("y: ", y)
    print("w: ", w)
    print("h: ", h)
    print("bound: ", bound)
    print("leap: ", leap)
    print("num_notes: ", num_notes)
    print("num_area: ", num_area)

first_frame shape:  (2118, 2068, 3)
x:  1059
y:  50
w:  0
h:  880
bound:  0
leap:  3
num_notes:  88
num_area:  10


In [23]:
def map_value(value, min_value, max_value, min_result, max_result):
    '''maps value (or array of values) from one range to another'''
    
    result = min_result + (value - min_value)/(max_value - min_value)*(max_result - min_result)
    return result

In [15]:
prev_frame_cnt = 0

while(True):
    magnitude = []
    age = []
    start = time.time()
    ret, frame = cap.read()
    if not ret:
        print('video error')
        cv.waitKey(1)
        cv.destroyAllWindows()
        cv.waitKey(1)
        break
    frame_cnt += 1
    roi = frame[y:y+h, x:x+1]

    roi_gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)

    accumulated_pixel_color = 0
    if frame_cnt % leap == 0:
        for i in range(h): 
            if i is (h-1):
                cnt = i % num_area
                if((accumulated_pixel_color // cnt) < bound):
                    accumulated_pixel_color = bound
                avg_color = accumulated_pixel_color // cnt
                mag = avg_color
                # normalize function
                # NOTE: Modify this.
                # mag = mag ** 2 // 100
                magnitude.append(mag)
                age.append(frame_cnt)
                accumulated_pixel_color = 0
            elif (i % num_area) is (num_area - 1):
                # if(printFlag):
                #     print(frame_cnt)
                # white = 255, black = 0. if whiter than 200, cap it to max_white
                if((accumulated_pixel_color // num_area) < bound):
                    accumulated_pixel_color = 0
                avg_color = accumulated_pixel_color // num_area
                mag = avg_color
                # normalize function
                # NOTE: Modify this.
                # mag = mag ** 2 // 100
                magnitude.append(mag)
                age.append(frame_cnt)
                accumulated_pixel_color = 0
            else:
                accumulated_pixel_color += (255 - roi_gray[i][0])
                # if(printFlag):
                    # print(roi_gray[i][0])
        dict = {'magnitude': magnitude, 'age': age}
    
        df = pd.DataFrame(dict)

        n_impacts = len(df)
        # print(n_impacts, 'impacts')

        df.head() #take a look at first 5 rows
        
        ages = df['age'].values    #this is a numpy array (not a list), you can do mathematical operations directly on the object
        # print(ages)
        magnitudes = df['magnitude'].values 
        indexes = df.index.values
        # print(indexes)

        # times_myrs = max(ages) - ages  #measure time from oldest crater (first impact) in data
        times_myrs = ages

        myrs_per_beat = 60  #number of Myrs for each beat of music 
        t_data = times_myrs/myrs_per_beat #rescale time from Myrs to beats
        duration_beats = max(t_data)  #duration in beats (actually, onset of last note)
        print('Duration:', duration_beats, 'beats')
            
            
        norm_magnitude = map_value(magnitudes, min(magnitudes), max(magnitudes), 0, 1) #normalize data, so it runs from 0 to 1 
        norm_scale = 1  #lower than 1 to spread out more evenly
        norm_magnitude = norm_magnitude**norm_scale

        vel_min,vel_max = 0, 127   #minimum and maximum note velocity
        midi_data = []
        vel_data = []
        print(n_impacts)
        for i in range(n_impacts):
            # note_index = round(map_value(y_data[i], 0, 1, 0, num_notes-1))
            note_index = indexes[i] % num_notes
            midi_data.append(note_midis[note_index])

            note_velocity = round(map_value(norm_magnitude[i]**1, 0, 1, vel_min, vel_max)) #bigger craters will be louder
            #we round here because note velocites are integers
            vel_data.append(note_velocity * 1)


            # client.send_message("/note", midi_data[(x)*88:(x+1)*88])
            # client.send_message("/velocity", vel_data[(x)*88:(x+1)*88])
            client.send_message("/note", note_midis[note_index])
            client.send_message("/velocity", note_velocity)
            time.sleep(0.125)
    
        mid = time.time()
        print("time taken between 1)~ 3-1): " + f"{mid - start:.5f} sec")
    
        #create midi file object, add tempo
        my_midi_file = MIDIFile(frame_cnt) #one track 
        my_midi_file.addTempo(track=0, time=0, tempo=tp) 
        #add midi notes
        for i in range(n_impacts):
            my_midi_file.addNote(track=0, channel=0, pitch=midi_data[i], time=t_data[i], duration=0.5\
                                 , volume=vel_data[i])

        filename = "./saved/" + str(video) + "_" + str(scale_selection)+ "_" + str(leap) + "/" + str(frame_cnt)

        #create and save the midi file itself
        with open(filename + '.mid', "wb") as f:
            my_midi_file.writeFile(f) 

        
        mid2 = time.time()
        print("time taken in saving midi: " + f"{mid2 - mid:.5f} sec")
        
        midifile = filename + '.mid'
        
        
        end = time.time()
        prev_frame_cnt = frame_cnt
        print("time taken in playing midi: " + f"{end - mid2:.5f} sec")
        if (frame_cnt == 49):
            break
        # break
            
        if cv.waitKey(1) & 0xff == ord('q'):
            break

while(True):
    # print(frame_cnt)
    magnitude = []
    age = []
    start = time.time()
    ret, frame = cap.read()
    if not ret:
        print('video error')
        cv.waitKey(1)
        cv.destroyAllWindows()
        cv.waitKey(1)
        break
    frame_cnt += 1
    roi = frame[y:y+h, x:x+1]
    # if(printFlag):
        # print(np.shape(roi))
    roi_gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
    # if(printFlag):
        # print(roi_gray)
    # print(np.shape(roi_gray))
    accumulated_pixel_color = 0
    if frame_cnt % leap == 0:
        for i in range(h): 
            if i is (h-1):
                # if(printFlag):
                    # print(frame_cnt)
                cnt = i % num_area
                if((accumulated_pixel_color // cnt) < bound):
                    accumulated_pixel_color = bound
                avg_color = accumulated_pixel_color // cnt
                mag = avg_color
                # normalize function
                # NOTE: Modify this.
                # mag = mag ** 2 // 100
                magnitude.append(mag)
                age.append(frame_cnt)
                accumulated_pixel_color = 0
            elif (i % num_area) is (num_area - 1):
                # if(printFlag):
                #     print(frame_cnt)
                # white = 255, black = 0. if whiter than 200, cap it to max_white
                if((accumulated_pixel_color // num_area) < bound):
                    accumulated_pixel_color = 0
                avg_color = accumulated_pixel_color // num_area
                mag = avg_color
                # normalize function
                # NOTE: Modify this.
                # mag = mag ** 2 // 100
                magnitude.append(mag)
                age.append(frame_cnt)
                accumulated_pixel_color = 0
            else:
                accumulated_pixel_color += (255 - roi_gray[i][0])

        dict = {'magnitude': magnitude, 'age': age}
    
        df = pd.DataFrame(dict)
        n_impacts = len(df)

        df.head() #take a look at first 5 rows
        
        ages = df['age'].values    #this is a numpy array (not a list), you can do mathematical operations directly on the object
        magnitudes = df['magnitude'].values 
        indexes = df.index.values

        times_myrs = ages


        myrs_per_beat = 60  #number of Myrs for each beat of music 
        t_data = times_myrs/myrs_per_beat #rescale time from Myrs to beats
        duration_beats = max(t_data)  #duration in beats (actually, onset of last note)
        print('Duration:', duration_beats, 'beats')
            
            
        norm_magnitude = map_value(magnitudes, min(magnitudes), max(magnitudes), 0, 1) #normalize data, so it runs from 0 to 1 
        norm_scale = 1  #lower than 1 to spread out more evenly
        norm_magnitude = norm_magnitude**norm_scale

        midi_data = []
        for i in range(n_impacts):
            # note_index = round(map_value(y_data[i], 0, 1, 0, num_notes-1)) 
            note_index = indexes[i] % num_notes
            midi_data.append(note_midis[note_index])

            
        vel_min,vel_max =0,127   #minimum and maximum note velocity

        vel_data = []
        for i in range(n_impacts):
            note_velocity = round(map_value(norm_magnitude[i]**1, 0, 1, vel_min, vel_max)) #bigger craters will be louder
            #we round here because note velocites are integers
            vel_data.append(note_velocity * 1)

        # plt.rcParams['figure.facecolor'] = 'white'
        # plt.scatter(t_data, midi_data, s=vel_data, c = '#FFFFFF')
        # plt.title('midi sheet (vel data applied)')
        # plt.scatter(t_data, midi_data, s=vel_data)
        # plt.xlabel('time [beats]')
        # plt.ylabel('midi note numbers')
        # plt.show()
        # plt.savefig(newFolderName+'/midi_vel.png')
            
        # from midiutil import MIDIFile #import library to make midi file, https://midiutil.readthedocs.io/en/1.2.1/
        
        mid = time.time()
        print("time taken between 1)~ 3-1): " + f"{mid - start:.5f} sec")
    
        #create midi file object, add tempo
        my_midi_file = MIDIFile(frame_cnt) #one track 
        my_midi_file.addTempo(track=0, time=0, tempo=tp) 
        #add midi notes
        for i in range(n_impacts):
            my_midi_file.addNote(track=0, channel=0, pitch=midi_data[i], time=t_data[i], duration=0.5\
                                 , volume=vel_data[i])

        filename = "./saved/" + str(video) + "_" + str(scale_selection)+ "_" + str(leap) + "/" + str(frame_cnt)

        #create and save the midi file itself
        with open(filename + '.mid', "wb") as f:
            my_midi_file.writeFile(f) 

        
        mid2 = time.time()
        print("time taken in saving midi: " + f"{mid2 - mid:.5f} sec")
        
        midifile = filename + '.mid'
        


        
        end = time.time()
        prev_frame_cnt = frame_cnt


            
        if cv.waitKey(1) & 0xff == ord('q'):
            break
        
print("calculation done") 
# dict = {'magnitude': magnitude, 'age': age}
# if(printFlag):
    # print(dict)
cv.destroyAllWindows()
cv.waitKey(1)

# The following frees up resources and closes all windows
cap.release()
cv.destroyAllWindows()

video size in pixel:  (2118, 2068, 3)
first_frame shape:  (2118, 2068, 3)
x:  1059
y:  50
w:  0
h:  880
bound:  0
leap:  3
num_notes:  88
num_area:  10
Duration: 0.05 beats
88
time taken between 1)~ 3-1): 11.35146 sec


FileNotFoundError: [Errno 2] No such file or directory: './saved/test.mp4_1_3/3.mid'

In [19]:
import pretty_midi
# Load MIDI file into PrettyMIDI object
midi_data = pretty_midi.PrettyMIDI(midifile)
cello_program = pretty_midi.instrument_name_to_program('Cello')
cello = pretty_midi.Instrument(program=cello_program)
midi_data.instruments.append(cello)
new_filename = 'cellomidi'
new_midifile = newFolderName + "/" + filename + '.mid'
midi_data.write(new_midifile)

In [16]:
# example usage for creating a simple MIDI file:

# import pretty_midi
# # Create a PrettyMIDI object
# cello_c_chord = pretty_midi.PrettyMIDI()
# # Create an Instrument instance for a cello instrument
# cello_program = pretty_midi.instrument_name_to_program('Cello')
# cello = pretty_midi.Instrument(program=cello_program)
# # Iterate over note names, which will be converted to note number later
# for note_name in ['C5', 'E5', 'G5']:
#     # Retrieve the MIDI note number for this note name
#     note_number = pretty_midi.note_name_to_number(note_name)
#     # Create a Note instance for this note, starting at 0s and ending at .5s
#     note = pretty_midi.Note(velocity=100, pitch=note_number, start=0, end=.5)
#     # Add it to our cello instrument
#     cello.notes.append(note)
# # Add the cello instrument to the PrettyMIDI object
# cello_c_chord.instruments.append(cello)
# # Write out the MIDI data
# cello_c_chord.write('cello-C-chord.mid')